In [1]:
import json
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torch import Generator

In [2]:
from os.path import expanduser
home = expanduser("~")
path = home + "/vlp/dat/nli/XNLI-1.0/vi.tok.jsonl"
print(path)

/Users/phuonglh/vlp/dat/nli/XNLI-1.0/vi.tok.jsonl


In [3]:
class VietnameseXNLI(Dataset):
    def __init__(self, jsonlPath):
        self.X = []
        self.y = []
        with open(jsonlPath) as f:
            for line in f:
                sample = json.loads(line)
                self.X.append(sample["sentence1_tokenized"] +
                              " </s> " + sample["sentence2_tokenized"])
                self.y.append(sample["gold_label"])

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.X[i], self.y[i]


In [4]:
dataset = VietnameseXNLI(path)
N = int(0.8*len(dataset))
training, test = random_split(dataset, [N, len(dataset)-N], generator=Generator().manual_seed(12345))

In [117]:
batch_size=32

In [5]:
# for testing only
train_loader = DataLoader(training, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

In [6]:
next(iter(test_loader))

[('ừm không có gì sai với việc một phụ_huynh tặng tất_cả mọi thứ cô ấy có để ừ cho ừ cho một cá_nhân như ừ như bạn </s> Bố_mẹ có_thể cho con_cái nhiều quà_cáp .',
  'Ngày_nay , những người Đức này thậm_chí không ở lại Hoa_Kỳ . </s> Những người Đức này được sử_dụng trên khắp châu Mỹ ngày này .',
  'tôi sợ là , tôi nghĩ rằng tên anh ấy_là Anderson , một quý ông đang tìm_kiếm một chiếc vé độc_lập chống lại Reagan </s> Anderson đánh_bại Reagan .',
  'Nếu có_khi nào tôi viết một cuốn tự_truyện , nó sẽ là từ_điển của những địa_danh và con_người được định_nghĩa theo tầm quan_trọng riêng . </s> Điều quan_trọng với tôi là tự_truyện của tôi có_thể được tiếp_cận dễ_dàng nhất .',
  'Trong xe_ngựa kéo , bạn ghé thăm dân_làng đang thu_hoạch , cắt lông cừu , mài bột trong nhà_máy , dệt , và làmt móng ngựa . </s> Người_dân trong làng ăn_mặc như thời thuộc_địa .',
  'Những người không giao_tiếp với các ngôn_ngữ liên_quan có_lẽ sẽ không có câu trả_lời cho những câu_hỏi tu_từ này , nhưng tôi cảm_thấy chắ

In [8]:
from transformers import AutoModel, AutoTokenizer

In [9]:
bert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
tokenizer

PreTrainedTokenizer(name_or_path='vinai/phobert-base', vocab_size=64000, model_max_len=256, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'})

In [11]:
bert

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [44]:
label_dict = {"entailment": 0, "neutral": 1, "contradiction": 2}

In [14]:
import torch.nn as nn
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [37]:
import transformers
from transformers import (
    BertPreTrainedModel,
    RobertaConfig
)
from transformers.models.roberta.modeling_roberta import (
    RobertaClassificationHead,
    RobertaConfig,
    RobertaModel,
)
import torchtext
from torch.nn import CrossEntropyLoss


In [39]:
class RobertaPhoBERTClassifier(BertPreTrainedModel):
    def __init__(self, config):
        super(RobertaPhoBERTClassifier, self).__init__(config)
        self.num_labels = 3
        self.model = RobertaModel(config)
        self.classifier = RobertaClassificationHead(config)

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        outputs = (loss,) + outputs
        return outputs  # (loss,) logits, (hidden_states, (attentions))


In [40]:
# https://pchanda.github.io/Roberta-FineTuning-for-Classification/
config_class = RobertaConfig
model_class = RobertaPhoBERTClassifier
config = config_class.from_pretrained("vinai/phobert-base", num_labels=3)
model = model_class.from_pretrained("vinai/phobert-base", config=config)

model

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaPhoBERTClassifier: ['roberta.encoder.layer.4.attention.output.LayerNorm.bias', 'roberta.encoder.layer.11.attention.self.value.bias', 'roberta.encoder.layer.2.attention.self.value.weight', 'roberta.encoder.layer.2.intermediate.dense.weight', 'roberta.encoder.layer.4.intermediate.dense.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.attention.self.value.weight', 'roberta.encoder.layer.7.attention.self.query.bias', 'roberta.encoder.layer.3.attention.self.value.weight', 'roberta.encoder.layer.7.attention.output.dense.bias', 'roberta.encoder.layer.6.attention.self.query.bias', 'roberta.encoder.layer.1.intermediate.dense.bias', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.8.output.dense.weight', 'roberta.encoder.layer.9.intermediate.dense.weight', 'roberta.encoder.layer.7.attention.self.key.bias', 'roberta.encoder.layer.9.ou

RobertaPhoBERTClassifier(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [93]:
class ClassificationDataset(Dataset):
    def __init__(self, data, tokenizer):
        xs, ys = data
        # this is like calling tokenizer.encode() but has paddings
        self.examples = tokenizer(text=xs, text_pair=None, truncation=True, padding='max_length',
                                  max_length=tokenizer.model_max_length, return_tensors='pt')
        self.labels = torch.tensor([label_dict[y] for y in ys], dtype=torch.long)

    def __len__(self):
        return len(self.examples["input_ids"])

    def __getitem__(self, index):
        return {key: self.examples[key][index] for key in self.examples}, self.labels[index]


In [78]:
# each data is a pair of (xs, ys)
training_data = ([x[0] for x in training], [x[1] for x in training])
test_data = ([x[0] for x in test], [x[1] for x in test])

In [80]:
training_ds = ClassificationDataset(training_data, tokenizer)
test_ds = ClassificationDataset(test_data, tokenizer)

In [96]:
# create data loader
train_loader = DataLoader(training_ds, batch_size=batch_size)
test_loader = DataLoader(test_ds, batch_size=batch_size)

In [98]:
first_test_batch = next(iter(test_loader))

In [101]:
# xs in the first batch
first_test_batch[0]

{'input_ids': tensor([[    0, 14583,   599,  ...,     1,     1,     1],
         [    0,  6177,     4,  ...,     1,     1,     1],
         [    0,    70,  1080,  ...,     1,     1,     1],
         ...,
         [    0,    92,   573,  ...,     1,     1,     1],
         [    0,   146,    69,  ...,     1,     1,     1],
         [    0,   404,    16,  ...,     1,     1,     1]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [104]:
# ys in the first batch
first_test_batch[1]

tensor([0, 1, 2, 2, 1, 0, 1, 2, 0, 2, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 0, 0, 0, 2,
        0, 0, 2, 1, 2, 1, 1, 0])

In [106]:
def get_inputs_dict(batch):
    inputs = {key: value.squeeze(1).to(DEVICE) for key, value in batch[0].items()}
    inputs["labels"] = batch[1].to(DEVICE)
    return inputs

In [107]:
get_inputs_dict(first_test_batch)

{'input_ids': tensor([[    0, 14583,   599,  ...,     1,     1,     1],
         [    0,  6177,     4,  ...,     1,     1,     1],
         [    0,    70,  1080,  ...,     1,     1,     1],
         ...,
         [    0,    92,   573,  ...,     1,     1,     1],
         [    0,   146,    69,  ...,     1,     1,     1],
         [    0,   404,    16,  ...,     1,     1,     1]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([0, 1, 2, 2, 1, 0, 1, 2, 0, 2, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 0, 0, 0, 2,
         0, 0, 2, 1, 2, 1, 1, 0])}

In [109]:
batch = get_inputs_dict(first_test_batch)
first_input_ids = batch['input_ids'].to(DEVICE)
first_attention_mask = batch['attention_mask'].to(DEVICE)
first_labels = batch['labels'].to(DEVICE)

In [110]:
print(batch)

{'input_ids': tensor([[    0, 14583,   599,  ...,     1,     1,     1],
        [    0,  6177,     4,  ...,     1,     1,     1],
        [    0,    70,  1080,  ...,     1,     1,     1],
        ...,
        [    0,    92,   573,  ...,     1,     1,     1],
        [    0,   146,    69,  ...,     1,     1,     1],
        [    0,   404,    16,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 2, 2, 1, 0, 1, 2, 0, 2, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 0, 0, 0, 2,
        0, 0, 2, 1, 2, 1, 1, 0])}


In [112]:
# for testing
model(first_input_ids, first_attention_mask, first_labels)

(tensor(1.0939, grad_fn=<NllLossBackward0>),
 tensor([[0.2798, 0.1798, 0.2442],
         [0.2941, 0.1732, 0.3278],
         [0.2628, 0.2264, 0.3528],
         [0.2902, 0.1886, 0.3668],
         [0.2658, 0.2300, 0.3313],
         [0.2819, 0.1992, 0.3798],
         [0.3302, 0.1284, 0.3000],
         [0.3033, 0.1471, 0.3298],
         [0.2693, 0.1761, 0.3628],
         [0.2615, 0.1826, 0.3168],
         [0.2951, 0.1279, 0.2781],
         [0.2566, 0.2005, 0.3472],
         [0.2596, 0.1991, 0.3500],
         [0.2887, 0.1695, 0.2419],
         [0.2949, 0.1784, 0.3382],
         [0.3207, 0.1255, 0.3589],
         [0.2571, 0.1994, 0.3595],
         [0.2897, 0.1694, 0.3244],
         [0.2506, 0.2077, 0.2737],
         [0.2984, 0.1981, 0.3337],
         [0.2868, 0.1829, 0.3259],
         [0.2960, 0.1705, 0.3220],
         [0.3104, 0.0864, 0.3696],
         [0.2798, 0.1910, 0.2853],
         [0.2929, 0.2194, 0.4174],
         [0.3161, 0.1978, 0.3310],
         [0.3340, 0.1649, 0.3496],
         [

In [113]:
import math
from transformers.optimization import (
    AdamW, 
    get_linear_schedule_with_warmup
)

from scipy.special import softmax
from torch.nn import CrossEntropyLoss

from sklearn.metrics import (
    confusion_matrix,
    matthews_corrcoef,
    roc_curve,
    auc,
    average_precision_score,
)

In [119]:
warmup_ratio = 0.06
weight_decay=0.0
gradient_accumulation_steps = 1
num_train_epochs = 15
learning_rate = 1e-05
adam_epsilon = 1e-08
t_total = len(train_loader) // gradient_accumulation_steps * num_train_epochs
optimizer_grouped_parameters = []
custom_parameter_names = set()
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters.extend(
    [
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if n not in custom_parameter_names and not any(nd in n for nd in no_decay)
            ],
            "weight_decay": weight_decay,
        },
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if n not in custom_parameter_names and any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
)

warmup_steps = math.ceil(t_total * warmup_ratio)
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

In [118]:
import numpy as np

def compute_metrics(preds, model_outputs, labels, eval_examples=None, multi_label=False):
    assert len(preds) == len(labels)
    mismatched = labels != preds
    wrong = [i for (i, v) in zip(eval_examples, mismatched) if v.any()]
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds, labels=[0, 1]).ravel()
    scores = np.array([softmax(element)[1] for element in model_outputs])
    fpr, tpr, thresholds = roc_curve(labels, scores)
    auroc = auc(fpr, tpr)
    auprc = average_precision_score(labels, scores)
    return (
        {
            **{"mcc": mcc, "tp": tp, "tn": tn, "fp": fp, "fn": fn, "auroc": auroc, "auprc": auprc},
        },
        wrong,
    )

def print_confusion_matrix(result):
    print('confusion matrix:')
    print('            predicted    ')
    print('          0     |     1')
    print('    ----------------------')
    print('   0 | ',format(result['tn'],'5d'),' | ',format(result['fp'],'5d'))
    print('gt -----------------------')
    print('   1 | ',format(result['fn'],'5d'),' | ',format(result['tp'],'5d'))
    print('---------------------------------------------------')

In [120]:
model.to(DEVICE)

model.zero_grad()

for epoch in range(num_train_epochs):

    model.train()
    epoch_loss = []
    
    for batch in train_loader:
        batch = get_inputs_dict(batch)
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        epoch_loss.append(loss.item())
        
    # evaluate model on test data at the end of the epoch.
    eval_loss = 0.0
    nb_eval_steps = 0
    n_batches = len(test_loader)
    preds = np.empty((len(test_ds), 3))
    out_label_ids = np.empty((len(test_ds)))
    model.eval()
    
    for i, test_batch in enumerate(test_loader):
        with torch.no_grad():
            test_batch = get_inputs_dict(test_batch)
            input_ids = test_batch['input_ids'].to(DEVICE)
            attention_mask = test_batch['attention_mask'].to(DEVICE)
            labels = test_batch['labels'].to(DEVICE)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            tmp_eval_loss, logits = outputs[:2]
            eval_loss += tmp_eval_loss.item()
            
        nb_eval_steps += 1
        start_index = batch_size * i
        end_index = start_index + batch_size if i != (n_batches - 1) else len(test_ds)
        preds[start_index:end_index] = logits.detach().cpu().numpy()
        out_label_ids[start_index:end_index] = test_batch["labels"].detach().cpu().numpy()
        
    eval_loss = eval_loss / nb_eval_steps
    model_outputs = preds
    preds = np.argmax(preds, axis=1)
    result, wrong = compute_metrics(preds, model_outputs, out_label_ids, test_data)
    
    print('epoch',epoch,'Training avg loss',np.mean(epoch_loss))
    print('epoch',epoch,'Testing  avg loss',eval_loss)
    print(result)
    print_confusion_matrix(result)
    print('---------------------------------------------------\n')